<a href="https://colab.research.google.com/github/Jaew00/Grad_PJT/blob/main/Pytorch_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1. Settings

> 1) Important required libraries




In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import os

In [2]:
import torch

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu') #GPU 할당

In [3]:
import random

def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed_everything(929)

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


2. Data Loader

트레이닝 데이터

In [97]:
import pandas as pd
raw_d_set = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/졸업 프로젝트/new_UrbanSound8K.csv')

In [98]:
raw_d_set.columns = ['file_name', 'fsID', 'start', 'end', 'salience', 'fold',
       'classID', 'class', 'label']

In [99]:
d_set = raw_d_set[['fsID', 'file_name', 'label']]
d_set

,fsID,file_name,label
0,100648,100648-1-0-0.wav,0
1,100648,100648-1-1-0.wav,0
2,100648,100648-1-2-0.wav,0
3,100648,100648-1-3-0.wav,0
4,100648,100648-1-4-0.wav,0
...,...,...,...
5353,96657,96657-8-0-1.wav,5
5354,96657,96657-8-0-2.wav,5
5355,96657,96657-8-0-3.wav,5
5356,98525,98525-8-0-0.wav,5


In [8]:
d_set['label'].value_counts()

1    1000
2    1000
3    1000
4    1000
5     929
0     429
Name: label, dtype: int64

In [9]:
import librosa

data, sample_rate = librosa.load('/content/drive/MyDrive/Colab Notebooks/졸업 프로젝트/train/46669-4-0-63.wav', sr = 16000)
print('sample_rate:', sample_rate, ', audio shape:', data.shape)
print('length:', data.shape[0]/float(sample_rate), 'secs')


sample_rate: 16000 , audio shape: (64000,)
length: 4.0 secs


In [10]:
def train_dataset():
    folder = "/content/drive/MyDrive/Colab Notebooks/졸업 프로젝트/train"
    dataset = []
    for file in tqdm(os.listdir(folder),colour='green'):
        if 'wav' in file:
            abs_file_path = os.path.join(folder,file)
            data, sr = librosa.load(abs_file_path, sr = 16000)
            class_label = int(d_set[d_set.file_name == file].label)
            dataset.append([data,class_label])
    
    print("Dataset 생성 완료")
    return pd.DataFrame(dataset,columns=['data','label'])

In [11]:
def test_dataset():
    folder = "/content/drive/MyDrive/Colab Notebooks/졸업 프로젝트/test"
    dataset = []
    for file in tqdm(os.listdir(folder),colour='green'):
        if 'wav' in file:
            abs_file_path = os.path.join(folder,file)
            data, sr = librosa.load(abs_file_path, sr = 16000)
            class_label = int(d_set[d_set.file_name == file].label)
            dataset.append([data, class_label])
    
    print("Dataset 생성 완료")
    return pd.DataFrame(dataset,columns=['data', 'label'])

In [12]:
train_wav = train_dataset()
test_wav = test_dataset()

100%|██████████| 3893/3893 [10:34<00:00,  6.14it/s]


Dataset 생성 완료


100%|██████████| 1467/1467 [03:40<00:00,  6.66it/s]

Dataset 생성 완료


In [13]:
train_wav.head()

,data,label
0,"[0.00021754522, 0.0006149989, 0.0003622311, 7....",2
1,"[-0.0010482423, -0.001591192, -0.0014650486, -...",2
2,"[0.0015225668, 0.002416549, 0.002141978, 0.002...",2
3,"[-0.012430043, 0.0076278527, 0.013844067, -0.0...",2
4,"[-0.00051125255, -0.00069866417, -0.0004001849...",2


In [14]:
train_wav['label'].value_counts()

4    744
3    730
2    700
1    700
5    684
0    334
Name: label, dtype: int64

In [15]:
test_wav.head()

,data,label
0,"[0.095636666, 0.10910979, -0.01036054, -0.0850...",5
1,"[0.022872224, 0.035138633, 0.038074523, 0.0550...",5
2,"[0.15510976, 0.22184339, 0.114296064, 0.050894...",5
3,"[0.16325384, 0.22990061, 0.09352431, -0.028997...",5
4,"[0.028184531, 0.06626986, 0.0696544, 0.0277310...",5


In [16]:
test_wav['label'].value_counts()

2    300
1    300
3    270
4    256
5    245
0     95
Name: label, dtype: int64

In [17]:
train_x = np.array(train_wav.data)
test_x = np.array(test_wav.data)

In [18]:
# 음성의 길이 중 가장 작은 길이를 구합니다.

def get_mini(data):

    mini = 9999999
    for i in data:
        if len(i) < mini:
            mini = len(i)

    return mini

train_mini = get_mini(train_x)
test_mini = get_mini(test_x)

#음성들의 길이를 맞춰줍니다.

mini = np.min([train_mini, test_mini])
print('가장 작은 길이 :', mini)

가장 작은 길이 : 800


In [19]:
def set_length(data, d_mini):

    result = []
    for i in data:
        result.append(i[:d_mini])
    result = np.array(result)

    return result

train_x = set_length(train_x, mini)
test_x = set_length(test_x, mini)

In [20]:
print('train :', train_x.shape)
print('test :', test_x.shape)

train : (3892, 800)
test : (1466, 800)


In [21]:
extracted_features = librosa.feature.mfcc(y=train_x[0], sr=16000, n_mfcc=40)
extracted_features.shape

/usr/local/lib/python3.7/dist-packages/librosa/core/spectrum.py:224: UserWarning: n_fft=2048 is too small for input signal of length=800
  n_fft, y.shape[-1]


(40, 2)

In [22]:
extracted_features_y = librosa.feature.mfcc(y=test_x[0], sr=16000, n_mfcc=40)
extracted_features_y.shape

(40, 2)

In [23]:
def preprocess_dataset(data):
    mfccs = []
    for i in data:
        extracted_features = librosa.feature.mfcc(y=i,
                                              sr=16000,
                                              n_mfcc=40)
        mfccs.append(extracted_features)
            
    return mfccs

In [24]:
train_mfccs = preprocess_dataset(train_x)
train_mfccs = np.array(train_mfccs)
train_mfccs = train_mfccs.reshape(-1, train_mfccs.shape[1], train_mfccs.shape[2], 1)
np.array(train_mfccs).shape

(3892, 40, 2, 1)

In [25]:
import torchvision.datasets as datasets # 데이터셋 집합체
import torchvision.transforms as transforms # 변환 툴

from torch.utils.data import DataLoader # 학습 및 배치로 모델에 넣어주기 위한 툴
from torch.utils.data import DataLoader, Dataset

class CustomDataset(Dataset):
    def __init__(self, X, y, train_mode=True, transforms=None): #필요한 변수들을 선언
        self.X = X
        self.y = y
        self.train_mode = train_mode
        self.transforms = transforms

    def __getitem__(self, index): #index번째 data를 return
        X = self.X[index]
        
        if self.transforms is not None:
            X = self.transforms(X)

        if self.train_mode:
            y = self.y[index]
            return X, y
        else:
            return X
    
    def __len__(self): #길이 return
        return len(self.X)

In [26]:
train_X = train_mfccs[:3300]
vali_X = train_mfccs[3300:]

In [27]:
train_y = train_wav.label[:3300]
vali_y = train_wav.label[3300:].reset_index(drop = True)

In [28]:
# 에포크 설정
num_epochs = 50

# 배치 사이즈 설정
batch_size = 10

#만든 train dataset를 DataLoader에 넣어 batch 만들기
train_dataset = CustomDataset(X=train_X, y=train_y)
train_loader = DataLoader(train_dataset, batch_size = batch_size, shuffle=True)

vali_dataset = CustomDataset(X=vali_X, y=vali_y)
vali_loader = DataLoader(vali_dataset, batch_size = batch_size, shuffle=False)

In [29]:
train_batches = len(train_loader)
vali_batches = len(vali_loader)

print('/ total train batches :', train_batches)
print('/ total valid batches :', vali_batches)

/ total train batches : 330
/ total valid batches : 60


In [30]:
from tqdm.auto import tqdm
import torch.nn.init as init
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
import torch.nn as nn # 신경망들이 포함됨

class CNN(torch.nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.layer = torch.nn.Sequential(
            nn.Conv2d(40, 16, kernel_size=2, stride=1, padding=1), #cnn layer
            nn.ELU(), #activation function
            #nn.MaxPool2d(kernel_size=2, stride=2), #pooling layer
            nn.BatchNorm2d(16),
            #nn.Dropout2d(p=0.2),

            nn.Conv2d(16, 32, kernel_size=2, stride=1, padding=1), #cnn layer
            nn.ELU(), #activation function
            #nn.BatchNorm2d(32),
            nn.MaxPool2d(kernel_size=2, stride=2), #pooling layer
            #nn.Dropout2d(p=0.2),

            nn.Conv2d(32, 64, kernel_size=2, stride=1, padding=1), #cnn layer
            nn.ELU(), #activation function
            #nn.BatchNorm2d(64),
            nn.MaxPool2d(kernel_size=2, stride=2), #pooling layer
            #nn.Dropout2d(p=0.2),

            nn.Conv2d(64, 128, kernel_size=2, stride=1, padding=1), #cnn layer
            nn.ELU(), #activation function
            nn.BatchNorm2d(128),
            nn.MaxPool2d(kernel_size=2, stride=2), #pooling layer
            nn.Dropout2d(0.2))
        
        self.fc_layer = nn.Sequential(
            nn.ELU(),
            nn.Linear(128, 6) #fully connected layer(ouput layer)
        )    


        for m in self.modules():
          if isinstance(m, nn.Conv2d):
            init.kaiming_normal_(m.weight.data) 
            m.bias.data.fill_(0)
          if isinstance(m, nn.Linear):
            init.kaiming_normal_(m.weight.data)
            m.bias.data.fill_(0)
        
    def forward(self, x):
        out = self.layer(x)
        out = torch.flatten(out, start_dim=1) # N차원 배열 -> 1차원 배열
        out = self.fc_layer(out)

        return out

In [31]:
import torch.optim as optim # 최적화 알고리즘들이 포함힘

model = CNN().to(device)
criterion = torch.nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(params = model.parameters(), lr = 1e-3 )
scheduler = None

In [32]:
from tqdm.auto import tqdm

def train(model, optimizer, train_loader, scheduler, device): 
    model.to(device)
    n = len(train_loader)
    best_acc = 0
    
    for epoch in range(1,num_epochs): #에포크 설정
        model.train() #모델 학습
        running_loss = 0.0
        
        for wav, label in tqdm(iter(train_loader)):
            
            wav, label = wav.to(device), label.to(device) #배치 데이터
            optimizer.zero_grad() #배치마다 optimizer 초기화
        
            # Data -> Model -> Output
            logit = model(wav) #예측값 산출
            loss = criterion(logit, label) #손실함수 계산
            
            # 역전파
            loss.backward() #손실함수 기준 역전파 
            optimizer.step() #가중치 최적화
            running_loss += loss.item()
             
        print('[%d] Train loss: %.10f' %(epoch, running_loss / len(train_loader)))
        
        if scheduler is not None:
            scheduler.step()
            
            
        #Validation set 평가
        model.eval() #evaluation 과정에서 사용하지 않아야 하는 layer들을 알아서 off 시키도록 하는 함수
        vali_loss = 0.0
        correct = 0
       
        with torch.no_grad(): #파라미터 업데이트 안하기 때문에 no_grad 사용
            for wav, label in tqdm(iter(vali_loader)):
                
                wav, label = wav.to(device), label.to(device)
                logit = model(wav)
                vali_loss += criterion(logit, label)
                pred = logit.argmax(dim=1, keepdim=True)  #10개의 class중 가장 값이 높은 것을 예측 label로 추출
                correct += pred.eq(label.view_as(pred)).sum().item() #예측값과 실제값이 맞으면 1 아니면 0으로 합산
        vali_acc = 100 * correct / len(vali_loader.dataset)
        print('Vail set: Loss: {:.4f}, Accuracy: {}/{} ( {:.0f}%)\n'.format(vali_loss / len(vali_loader), correct, len(vali_loader.dataset), 100 * correct / len(vali_loader.dataset)))
        
        #베스트 모델 저장
        if best_acc < vali_acc:
            best_acc = vali_acc
            
            torch.save(model.state_dict(), '/content/drive/MyDrive/Colab Notebooks/졸업 프로젝트/weights.best.cnn.pth') #이 디렉토리에 best_model.pth을 저장
            print('Model Saved.')

    print(best_acc)

In [33]:
train(model, optimizer, train_loader, scheduler, device)

  0%|          | 0/330 [00:00<?, ?it/s]

[1] Train loss: 1.5579171930


  0%|          | 0/60 [00:00<?, ?it/s]

Vail set: Loss: 1.1886, Accuracy: 332/592 ( 56%)

Model Saved.


  0%|          | 0/330 [00:00<?, ?it/s]

...

Vail set: Loss: 3.3466, Accuracy: 307/592 ( 52%)



  0%|          | 0/330 [00:00<?, ?it/s]

[49] Train loss: 0.1445758355


  0%|          | 0/60 [00:00<?, ?it/s]

Vail set: Loss: 2.6726, Accuracy: 313/592 ( 53%)

60.979729729729726


In [37]:
test_mfccs = preprocess_dataset(test_x)
test_mfccs = np.array(test_mfccs)
test_mfccs = test_mfccs.reshape(-1, test_mfccs.shape[1], test_mfccs.shape[2], 1)
np.array(test_mfccs).shape

/usr/local/lib/python3.7/dist-packages/librosa/core/spectrum.py:224: UserWarning: n_fft=2048 is too small for input signal of length=800
  n_fft, y.shape[-1]


(1466, 40, 2, 1)

In [38]:
test_X = test_mfccs
test_y = test_wav.label

In [42]:
def predict(model, test_loader, device):
    model.eval()
    model_pred = []
    correct_ = 0
    with torch.no_grad():
        for wav in tqdm(iter(test_loader)):
            wav = wav.to(device)

            pred_logit = model(wav)
            pred_logit = pred_logit.argmax(dim=1, keepdim=True).squeeze(1)

            model_pred.extend(pred_logit.tolist())
          
    return model_pred

In [43]:
test_dataset = CustomDataset(X=test_X, y=test_y, train_mode=False)
test_loader = DataLoader(test_dataset, batch_size = batch_size, shuffle=False)

In [55]:
# Validation Accuracy가 가장 뛰어난 모델을 불러옵니다.
checkpoint = torch.load('/content/drive/MyDrive/Colab Notebooks/졸업 프로젝트/weights.best.cnn.pth')
model = CNN().to(device)
model.load_state_dict(checkpoint)

# Inference
preds = predict(model, test_loader, device)


correct = 0
total = 0


total = len(test_wav.label)
correct += (preds == test_wav.label).sum()

acc = 100*correct/total
acc = float(acc)
print(preds[0:5], '\n', test_wav.label[0:5])
print(acc)

  0%|          | 0/147 [00:00<?, ?it/s]

[5, 5, 5, 5, 0] 
 0    5
1    5
2    5
3    5
4    5
Name: label, dtype: int64
55.66166439290587


In [56]:
def trans_data(sound_data):
    
    data, sr = librosa.load(sound_data, sr = 16000)
    data = np.array(data)
    print("Data Transfer Complete")

    return data

In [80]:
def trans_data():
    folder = "/content/drive/MyDrive/Colab Notebooks/졸업 프로젝트/testing"
    dataset = []
    for file in tqdm(os.listdir(folder),colour='green'):
        if 'wav' in file:
            abs_file_path = os.path.join(folder,file)
            data, sr = librosa.load(abs_file_path, sr = 16000)
            class_label = int(d_set[d_set.file_name == file].label)
            dataset.append([data, file])
    
    print("Dataset 생성 완료")
    return pd.DataFrame(dataset,columns=['data', 'file_name'])

In [131]:
trans_data = trans_data()

  0%|          | 0/1 [00:00<?, ?it/s]

Dataset 생성 완료


In [132]:
trans_data.head()
raw_data = trans_data

In [133]:
raw_data = np.array(raw_data.data)

In [91]:
mini = get_mini(raw_data)
print(mini)

data = set_length(raw_data, 800)
print(data.shape)

64000
(1, 800)


In [92]:
extracted_features = librosa.feature.mfcc(y=data[0], sr=16000, n_mfcc=40)
extracted_features.shape

/usr/local/lib/python3.7/dist-packages/librosa/core/spectrum.py:224: UserWarning: n_fft=2048 is too small for input signal of length=800
  n_fft, y.shape[-1]


(40, 2)

In [93]:
data_mfccs = preprocess_dataset(data)
data_mfccs = np.array(data_mfccs)
data_mfccs = data_mfccs.reshape(-1, data_mfccs.shape[1], data_mfccs.shape[2], 1)
np.array(data_mfccs).shape

(1, 40, 2, 1)

In [94]:
dataset = CustomDataset(X=data_mfccs, y= None, train_mode=False)
loader = DataLoader(dataset, batch_size = batch_size, shuffle=False)

In [174]:
# Validation Accuracy가 가장 뛰어난 모델을 불러옵니다.
checkpoint = torch.load('/content/drive/MyDrive/Colab Notebooks/졸업 프로젝트/weights.best.cnn.pth')
model = CNN().to(device)
model.load_state_dict(checkpoint)

# Inference
preds = predict(model, loader, device)
value = preds[0]
print('예측 Label은 {}'.format(value))
print('실제 label과 class는')
wav_data = trans_data['file_name'][0]
raw_d_set[raw_d_set['file_name'] == wav_data].loc[:,'class' : 'label']


  0%|          | 0/1 [00:00<?, ?it/s]

예측 Label은 4
실제 label과 class는


,class,label
4426,jackhammer,4
